In [31]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3
import ipywidgets as widgets
from IPython.display import clear_output

In [32]:
tradingList = {
    "HINDPETRO": {"price": 262.25, "leverage": 10},
    "HINDZINC": {"price": 317.10, "leverage": 10},
    "SBIN": {"price": 364.65, "leverage": 9},
    "ICICIBANK": {"price": 594.75, "leverage": 9},
    "WIPRO": {"price": 507.60, "leverage": 9},
    "ITC": {"price": 203.25, "leverage": 9},
    "BHARTIARTL": {"price": 561.75, "leverage": 9},
    "IOC": {"price": 102.80, "leverage": 9},
    "ONGC": {"price": 115.10, "leverage": 9},
    "DABUR": {"price": 529.20, "leverage": 9},
    "GAIL": {"price": 161.05, "leverage": 9},
    "TATACONSUM": {"price": 640.05, "leverage": 9},
    "AMBUJACEM": {"price": 309.05, "leverage": 9},
    "DLF": {"price": 265.30, "leverage": 8},
    "TATAPOWER": {"price": 109.35, "leverage": 8},
    "TATAMOTORS": {"price": 315.55, "leverage": 7}
}

tickerList = list(tradingList.keys())
db_hammer = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\hammer\\statistics.db"
data_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\hammer\\"

In [33]:
class Trend(enum.Enum):
    strong_uptrend = 1
    uptrend = 2
    strong_downtrend = 3
    downtrend = 4
    uptrend_reversal = 5
    downtrend_reversal = 6
    strong_uptrend_reversal = 7
    strong_downtrend_reversal = 8
    sidetrend = 9

In [34]:
class Candle(enum.Enum):
    red = 1
    green = 2
    doji = 3

In [43]:
class CandleHelper:

    def __init__(self, count=4):
        self.count = count
        self.index = 0

    def is_doji_candle(self, candle):
        return candle == Candle.doji

    def is_green_candle(self, candle):
        return candle == Candle.green

    def is_red_candle(self, candle):
        return candle == Candle.red
    
    def get_current_data(self,data):
        candle = self.get_last_candle(data)
        spread = 0
        head = 0
        tail = 0
        if self.is_red_candle(candle):
            spread = data['open'].values[0]  - data['close'].values[0] 
            head = data['high'].values[0]  - data['open'].values[0] 
            tail = data['close'].values[0]  - data['low'].values[0] 
        elif self.is_green_candle(candle):
            spread = data['close'].values[0]  - data['open'].values[0] 
            head = data['high'].values[0]  - data['close'].values[0] 
            tail = data['open'].values[0]  - data['low'].values[0] 
        else:
            spread = data['close'].values[0]  - data['open'].values[0] 
            head = data['high'].values[0]  - data['close'].values[0] 
            tail = data['open'].values[0]  - data['low'].values[0]            
            
        return candle.value,round(spread,2),round(head,2),round(tail,2)

    def get_last_candle(self, data):
        if data['close'].values > data['open'].values:
            return Candle.green
        if data['close'].values < data['open'].values:
            return Candle.red
        return Candle.doji

    def get_candle_details(self, data):
        candle_data = data.tail(4)
        current = self.get_last_candle(candle_data.tail(1))
        previous = self.get_last_candle(candle_data.tail(2).head(1))
        last_two = self.get_last_candle(candle_data.tail(3).head(1))
        last_three = self.get_last_candle(candle_data.head(1))
        return current, previous, last_two, last_three

    def trend_strength(self, data, trend):
        candle_data = data.tail(4)
        current = candle_data.tail(1)
        previous = candle_data.tail(2).head(1)
        last_two = candle_data.tail(3).head(1)
        last_three = candle_data.head(1)
        reversal_value = 0
        if trend == Trend.strong_uptrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.downtrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.downtrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_uptrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        else:
            trend_value = 0
            reversal_value = 0
        return trend_value, reversal_value

    def get_trends(self, data):
        current, previous, last_two, last_three = self.get_candle_details(data)
        trend = Trend.sidetrend
        if (self.is_green_candle(current) or self.is_doji_candle(current)) and (
                self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend
                else:
                    trend = Trend.uptrend
        elif (self.is_red_candle(current) or self.is_doji_candle(current)) and (
                self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend
                else:
                    trend = Trend.downtrend
        elif self.is_red_candle(current) and (self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend_reversal
                else:
                    trend = Trend.uptrend_reversal
        elif self.is_green_candle(current) and (self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend_reversal
                else:
                    trend = Trend.downtrend_reversal
        else:
            trend = Trend.sidetrend
        trend_value, reversal_value = self.trend_strength(data, trend)
        return trend.value, trend_value, reversal_value

    def get_next_candle(self,data):
        candle_data = data.tail(1)
        self.index +=1
        return self.get_last_candle(candle_data)
    

In [44]:
file_name = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\last_60\\17_05_2021\\WIPRO_17_05_2021_last_60.csv"
data = pd.read_csv(file_name)


In [45]:
d = Candle.red
d
d.value


1

In [46]:
data
candle_data = data.loc[5:5]
candle_data

,open,high,low,close,volume
5,420.0,420.4,419.4,419.8,55425


In [47]:
helper = CandleHelper()
trade_data = pd.DataFrame(columns = ['current_open','current_high','current_low','current_close','current_volume'])
data_length = len(data)
for index, row in data.iterrows():
    row_data = {}
    if index >= 3:
        candle_data = data.loc[index-3:index]
        trend, trend_value, reversal_value = helper.get_trends(candle_data)
        row_data["current_trend"] = trend
        row_data["current_trend_value"]= trend_value
        row_data["current_reversal_value"] = reversal_value
        
        candle,spread,head,tail = helper.get_current_data(data.loc[index:index])
        row_data["current_candle"] =candle
        row_data["current_spread"] =spread
        row_data["current_head"] =head
        row_data["current_tail"] =tail
        
        row_data["current_open"] = row["open"]
        row_data["current_high"] = row["high"]
        row_data["current_low"] = row["low"]
        row_data["current_close"]= row["close"]
        row_data["current_volume"] = row["volume"]
        trade_data = trade_data.append(row_data, ignore_index=True)
    
    
trade_data.to_csv("E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\last_60\\17_05_2021\\current_data\\WIPRO_17_05_2021_last_60.csv", index=index)

In [54]:
trade_data

,current_open,current_high,current_low,current_close,current_volume,current_candle,current_head,current_reversal_value,current_spread,current_tail,current_trend,current_trend_value
0,420.70,420.75,419.25,420.45,118783.0,1.0,0.05,0.00,0.25,1.20,3.0,1.05
1,420.50,421.00,420.00,420.00,50456.0,1.0,0.50,0.00,0.50,0.00,3.0,0.90
2,420.00,420.40,419.40,419.80,55425.0,1.0,0.40,0.00,0.20,0.40,3.0,1.10
3,419.80,419.85,419.50,419.70,34038.0,1.0,0.05,0.00,0.10,0.20,3.0,0.65
4,419.85,420.25,419.65,419.95,46487.0,2.0,0.30,0.15,0.10,0.20,8.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...
14242,499.80,500.00,499.75,499.90,29410.0,2.0,0.10,0.00,0.10,0.05,9.0,0.00
14243,499.90,499.90,499.65,499.65,20821.0,1.0,0.00,0.15,0.25,0.00,5.0,0.00
14244,499.65,500.20,499.55,500.10,48522.0,2.0,0.10,0.00,0.45,0.10,9.0,0.00
14245,500.15,500.40,499.60,499.70,46139.0,1.0,0.25,0.00,0.45,0.10,9.0,0.00


In [59]:

final_data = pd.DataFrame(columns = ['current_open','current_high','current_low','current_close','current_volume'])
data_length = len(final_data)
for index, row in trade_data.iterrows():
    row_data = {}
    if index >= 3 :
        
        candle_data = trade_data.loc[index-3:index]       
        previous = candle_data.tail(2).head(1)
        last_two = candle_data.tail(3).head(1)
        last_three = candle_data.head(1)  
        
      
        #current
        row_data["current_open"] = row["current_open"]
        row_data["current_high"] = row["current_high"]
        row_data["current_low"] = row["current_low"]
        row_data["current_close"]= row["current_close"]
        row_data["current_volume"] = row["current_volume"]
        row_data["current_candle"] =row["current_candle"]
        row_data["current_spread"] =row["current_spread"]
        row_data["current_head"] =row["current_head"]
        row_data["current_tail"] =row["current_tail"]
        row_data["current_trend"] = row["current_trend"]
        row_data["current_trend_value"]= row["current_trend_value"]
        row_data["current_reversal_value"] = row["current_reversal_value"]        
        
        #previous
        row_data["previous_open"] = previous["current_open"].values[0]
        row_data["previous_high"] = previous["current_high"].values[0]
        row_data["previous_low"] = previous["current_low"].values[0]
        row_data["previous_close"]= previous["current_close"].values[0]
        row_data["previous_volume"] = previous["current_volume"].values[0]
        row_data["previous_candle"] =previous["current_candle"].values[0]
        row_data["previous_spread"] =previous["current_spread"].values[0]
        row_data["previous_head"] =previous["current_head"].values[0]
        row_data["previous_tail"] =previous["current_tail"].values[0]
        row_data["previous_trend"] = previous["current_trend"].values[0]
        row_data["previous_trend_value"]= previous["current_trend_value"].values[0]
        row_data["previous_reversal_value"] = previous["current_reversal_value"].values[0]        
        
        #last_two
        row_data["last_two_open"] = last_two["current_open"].values[0]
        row_data["last_two_high"] = last_two["current_high"].values[0]
        row_data["last_two_low"] = last_two["current_low"].values[0]
        row_data["last_two_close"]= last_two["current_close"].values[0]
        row_data["last_two_volume"] = last_two["current_volume"].values[0]
        row_data["last_two_candle"] =last_two["current_candle"].values[0]
        row_data["last_two_spread"] =last_two["current_spread"].values[0]
        row_data["last_two_head"] =last_two["current_head"].values[0]
        row_data["last_two_tail"] =last_two["current_tail"].values[0]
        row_data["last_two_trend"] = last_two["current_trend"].values[0]
        row_data["last_two_trend_value"]= last_two["current_trend_value"].values[0]
        row_data["last_two_reversal_value"] = last_two["current_reversal_value"].values[0]        
        
        #last_three
        row_data["last_three_open"] = last_three["current_open"].values[0]
        row_data["last_three_high"] = last_three["current_high"].values[0]
        row_data["last_three_low"] = last_three["current_low"].values[0]
        row_data["last_three_close"]= last_three["current_close"].values[0]
        row_data["last_three_volume"] = last_three["current_volume"].values[0]
        row_data["last_three_candle"] =last_three["current_candle"].values[0]
        row_data["last_three_spread"] =last_three["current_spread"].values[0]
        row_data["last_three_head"] =last_three["current_head"].values[0]
        row_data["last_three_tail"] =last_three["current_tail"].values[0]
        row_data["last_three_trend"] = last_three["current_trend"].values[0]
        row_data["last_three_trend_value"]= last_three["current_trend_value"].values[0]
        row_data["last_three_reversal_value"] = last_three["current_reversal_value"].values[0]  
        
        
        
        #appending data
        final_data = final_data.append(row_data, ignore_index=True)
        
final_data.to_csv("E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\last_60\\17_05_2021\\final_data\\WIPRO_17_05_2021_last_60.csv", index=index)

In [60]:
final_data

,current_open,current_high,current_low,current_close,current_volume,current_candle,current_head,current_reversal_value,current_spread,current_tail,...,previous_head,previous_high,previous_low,previous_open,previous_reversal_value,previous_spread,previous_tail,previous_trend,previous_trend_value,previous_volume
0,419.80,419.85,419.50,419.70,34038.0,1.0,0.05,0.00,0.10,0.20,...,0.40,420.40,419.40,420.00,0.00,0.20,0.40,3.0,1.10,55425.0
1,419.85,420.25,419.65,419.95,46487.0,2.0,0.30,0.15,0.10,0.20,...,0.05,419.85,419.50,419.80,0.00,0.10,0.20,3.0,0.65,34038.0
2,419.90,420.00,419.60,419.90,30486.0,3.0,0.10,0.00,0.00,0.30,...,0.30,420.25,419.65,419.85,0.15,0.10,0.20,8.0,0.20,46487.0
3,419.90,419.90,418.60,418.90,51150.0,1.0,0.00,0.00,1.00,0.30,...,0.10,420.00,419.60,419.90,0.00,0.00,0.30,9.0,0.00,30486.0
4,418.95,418.95,418.40,418.40,52256.0,1.0,0.00,0.00,0.55,0.00,...,0.00,419.90,418.60,419.90,0.00,1.00,0.30,9.0,0.00,51150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14239,499.80,500.00,499.75,499.90,29410.0,2.0,0.10,0.00,0.10,0.05,...,0.00,499.80,499.60,499.70,0.00,0.10,0.10,9.0,0.00,41282.0
14240,499.90,499.90,499.65,499.65,20821.0,1.0,0.00,0.15,0.25,0.00,...,0.10,500.00,499.75,499.80,0.00,0.10,0.05,9.0,0.00,29410.0
14241,499.65,500.20,499.55,500.10,48522.0,2.0,0.10,0.00,0.45,0.10,...,0.00,499.90,499.65,499.90,0.15,0.25,0.00,5.0,0.00,20821.0
14242,500.15,500.40,499.60,499.70,46139.0,1.0,0.25,0.00,0.45,0.10,...,0.10,500.20,499.55,499.65,0.00,0.45,0.10,9.0,0.00,48522.0


In [62]:
 final_data
    

,current_open,current_high,current_low,current_close,current_volume,current_candle,current_head,current_reversal_value,current_spread,current_tail,...,previous_head,previous_high,previous_low,previous_open,previous_reversal_value,previous_spread,previous_tail,previous_trend,previous_trend_value,previous_volume
0,419.80,419.85,419.50,419.70,34038.0,1.0,0.05,0.00,0.10,0.20,...,0.40,420.40,419.40,420.00,0.00,0.20,0.40,3.0,1.10,55425.0
1,419.85,420.25,419.65,419.95,46487.0,2.0,0.30,0.15,0.10,0.20,...,0.05,419.85,419.50,419.80,0.00,0.10,0.20,3.0,0.65,34038.0
2,419.90,420.00,419.60,419.90,30486.0,3.0,0.10,0.00,0.00,0.30,...,0.30,420.25,419.65,419.85,0.15,0.10,0.20,8.0,0.20,46487.0
3,419.90,419.90,418.60,418.90,51150.0,1.0,0.00,0.00,1.00,0.30,...,0.10,420.00,419.60,419.90,0.00,0.00,0.30,9.0,0.00,30486.0
4,418.95,418.95,418.40,418.40,52256.0,1.0,0.00,0.00,0.55,0.00,...,0.00,419.90,418.60,419.90,0.00,1.00,0.30,9.0,0.00,51150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14239,499.80,500.00,499.75,499.90,29410.0,2.0,0.10,0.00,0.10,0.05,...,0.00,499.80,499.60,499.70,0.00,0.10,0.10,9.0,0.00,41282.0
14240,499.90,499.90,499.65,499.65,20821.0,1.0,0.00,0.15,0.25,0.00,...,0.10,500.00,499.75,499.80,0.00,0.10,0.05,9.0,0.00,29410.0
14241,499.65,500.20,499.55,500.10,48522.0,2.0,0.10,0.00,0.45,0.10,...,0.00,499.90,499.65,499.90,0.15,0.25,0.00,5.0,0.00,20821.0
14242,500.15,500.40,499.60,499.70,46139.0,1.0,0.25,0.00,0.45,0.10,...,0.10,500.20,499.55,499.65,0.00,0.45,0.10,9.0,0.00,48522.0


In [63]:

output_data = pd.DataFrame(columns = ['current_open','current_high','current_low','current_close','current_volume'])
data_length = len(final_data)
for index, row in final_data.iterrows():
    row_data = {}
    if index <= data_length-2:
        
        next_data = final_data.loc[index+1:index+1]      
 
      
        #current
        row_data["current_open"] = row["current_open"]
        row_data["current_high"] = row["current_high"]
        row_data["current_low"] = row["current_low"]
        row_data["current_close"]= row["current_close"]
        row_data["current_volume"] = row["current_volume"]
        row_data["current_candle"] =row["current_candle"]
        row_data["current_spread"] =row["current_spread"]
        row_data["current_head"] =row["current_head"]
        row_data["current_tail"] =row["current_tail"]
        row_data["current_trend"] = row["current_trend"]
        row_data["current_trend_value"]= row["current_trend_value"]
        row_data["current_reversal_value"] = row["current_reversal_value"]        
        
        #previous
        row_data["previous_open"] = row["current_open"]
        row_data["previous_high"] = row["current_high"]
        row_data["previous_low"] = row["current_low"]
        row_data["previous_close"]= row["current_close"]
        row_data["previous_volume"] = row["current_volume"]
        row_data["previous_candle"] =row["current_candle"]
        row_data["previous_spread"] =row["current_spread"]
        row_data["previous_head"] =row["current_head"]
        row_data["previous_tail"] =row["current_tail"]
        row_data["previous_trend"] = row["current_trend"]
        row_data["previous_trend_value"]= row["current_trend_value"]
        row_data["previous_reversal_value"] = row["current_reversal_value"]      
        
        #last_two
        row_data["last_two_open"] = row["current_open"]
        row_data["last_two_high"] = row["current_high"]
        row_data["last_two_low"] = row["current_low"]
        row_data["last_two_close"]= row["current_close"]
        row_data["last_two_volume"] = row["current_volume"]
        row_data["last_two_candle"] =row["current_candle"]
        row_data["last_two_spread"] =row["current_spread"]
        row_data["last_two_head"] =row["current_head"]
        row_data["last_two_tail"] =row["current_tail"]
        row_data["last_two_trend"] = row["current_trend"]
        row_data["last_two_trend_value"]= row["current_trend_value"]
        row_data["last_two_reversal_value"] = row["current_reversal_value"]    
        
        #last_three
        row_data["last_three_open"] = row["current_open"]
        row_data["last_three_high"] = row["current_high"]
        row_data["last_three_low"] = row["current_low"]
        row_data["last_three_close"]= row["current_close"]
        row_data["last_three_volume"] = row["current_volume"]
        row_data["last_three_candle"] =row["current_candle"]
        row_data["last_three_spread"] =row["current_spread"]
        row_data["last_three_head"] =row["current_head"]
        row_data["last_three_tail"] =row["current_tail"]
        row_data["last_three_trend"] = row["current_trend"]
        row_data["last_three_trend_value"]= row["current_trend_value"]
        row_data["last_three_reversal_value"] = row["current_reversal_value"]
        
        row_data["next_candle"] = next_data["current_candle"].values[0] 
        
        #appending data
        output_data = output_data.append(row_data, ignore_index=True)
        
output_data.to_csv("E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\last_60\\17_05_2021\\output_data\\WIPRO_17_05_2021_last_60.csv", index=index)

In [64]:
output_data

,current_open,current_high,current_low,current_close,current_volume,current_candle,current_head,current_reversal_value,current_spread,current_tail,...,previous_head,previous_high,previous_low,previous_open,previous_reversal_value,previous_spread,previous_tail,previous_trend,previous_trend_value,previous_volume
0,419.80,419.85,419.50,419.70,34038.0,1.0,0.05,0.00,0.10,0.20,...,0.05,419.85,419.50,419.80,0.00,0.10,0.20,3.0,0.65,34038.0
1,419.85,420.25,419.65,419.95,46487.0,2.0,0.30,0.15,0.10,0.20,...,0.30,420.25,419.65,419.85,0.15,0.10,0.20,8.0,0.20,46487.0
2,419.90,420.00,419.60,419.90,30486.0,3.0,0.10,0.00,0.00,0.30,...,0.10,420.00,419.60,419.90,0.00,0.00,0.30,9.0,0.00,30486.0
3,419.90,419.90,418.60,418.90,51150.0,1.0,0.00,0.00,1.00,0.30,...,0.00,419.90,418.60,419.90,0.00,1.00,0.30,9.0,0.00,51150.0
4,418.95,418.95,418.40,418.40,52256.0,1.0,0.00,0.00,0.55,0.00,...,0.00,418.95,418.40,418.95,0.00,0.55,0.00,4.0,0.95,52256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14238,499.70,499.80,499.60,499.80,41282.0,2.0,0.00,0.00,0.10,0.10,...,0.00,499.80,499.60,499.70,0.00,0.10,0.10,9.0,0.00,41282.0
14239,499.80,500.00,499.75,499.90,29410.0,2.0,0.10,0.00,0.10,0.05,...,0.10,500.00,499.75,499.80,0.00,0.10,0.05,9.0,0.00,29410.0
14240,499.90,499.90,499.65,499.65,20821.0,1.0,0.00,0.15,0.25,0.00,...,0.00,499.90,499.65,499.90,0.15,0.25,0.00,5.0,0.00,20821.0
14241,499.65,500.20,499.55,500.10,48522.0,2.0,0.10,0.00,0.45,0.10,...,0.10,500.20,499.55,499.65,0.00,0.45,0.10,9.0,0.00,48522.0


In [65]:
output_data.columns

Index(['current_open', 'current_high', 'current_low', 'current_close',
       'current_volume', 'current_candle', 'current_head',
       'current_reversal_value', 'current_spread', 'current_tail',
       'current_trend', 'current_trend_value', 'last_three_candle',
       'last_three_close', 'last_three_head', 'last_three_high',
       'last_three_low', 'last_three_open', 'last_three_reversal_value',
       'last_three_spread', 'last_three_tail', 'last_three_trend',
       'last_three_trend_value', 'last_three_volume', 'last_two_candle',
       'last_two_close', 'last_two_head', 'last_two_high', 'last_two_low',
       'last_two_open', 'last_two_reversal_value', 'last_two_spread',
       'last_two_tail', 'last_two_trend', 'last_two_trend_value',
       'last_two_volume', 'next_candle', 'previous_candle', 'previous_close',
       'previous_head', 'previous_high', 'previous_low', 'previous_open',
       'previous_reversal_value', 'previous_spread', 'previous_tail',
       'previous_trend',